In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from segmentation_models import Unet
from segmentation_models.losses import bce_jaccard_loss
from segmentation_models.metrics import iou_score
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import cv2

Segmentation Models: using `keras` framework.


In [3]:
metadata_path = '../data/HAM10000_metadata.csv'
metadata = pd.read_csv(metadata_path)

In [4]:
images_path = '../data\skin-cancer-mnist-ham10000_combined'  
masks_path = '../data\HAM10000_segmentations_lesion_tschandl'    

In [5]:
# Load image file names and labels
metadata['image_id'] = metadata['image_id'].apply(lambda x: x + '.jpg')
image_files = metadata['image_id'].values
labels = metadata['dx'].astype('category').cat.codes.values  # Convert labels to integer codes for 7 classes


In [6]:
images = []
masks = []
for img in image_files:
    img_path = os.path.join(images_path, img)
    msk = img.split(".")[0]+"_segmentation."+img.split(".")[1]
    mask_path = os.path.join(masks_path, msk.replace('.jpg', '.png'))
    if os.path.exists(img_path) and os.path.exists(mask_path):
        image = cv2.imread(img_path)
        image = cv2.resize(image, (224, 224))  # Resize to 224x224 for MobileNet input
        mask = cv2.imread(mask_path, 0)  # Load mask in grayscale
        mask = cv2.resize(mask, (224, 224))  # Resize mask to match image dimensions
        images.append(image)
        masks.append(mask)


In [12]:
len(images)

10015

In [11]:
images = np.array(images) / 255.0  # Normalize images
masks = np.expand_dims(np.array(masks) / 255.0, axis=-1)  # Normalize masks

In [40]:
print(f"Image shape: {images.shape}")    # Should be (num_samples, 224, 224, 3)
print(f"Mask shape: {masks.shape}")      # Should be (num_samples, 224, 224, 1)

Image shape: (10015, 224, 224, 3)
Mask shape: (10015, 224, 224, 1)


In [13]:
# Split data into training and validation sets
X_train, X_val, y_train, y_val, labels_train, labels_val = train_test_split(images, masks, labels, test_size=0.2, random_state=42)


In [14]:
# Data Augmentation for training
data_gen_args = dict(rotation_range=20,
                     width_shift_range=0.1,
                     height_shift_range=0.1,
                     shear_range=0.1,
                     zoom_range=0.2,
                     horizontal_flip=True,
                     fill_mode='nearest')

In [15]:
image_datagen = ImageDataGenerator(**data_gen_args)
mask_datagen = ImageDataGenerator(**data_gen_args)

In [16]:
# Generators for images and masks
train_image_generator = image_datagen.flow(X_train, batch_size=16, seed=42)
train_mask_generator = mask_datagen.flow(y_train, batch_size=16, seed=42)
val_image_generator = image_datagen.flow(X_val, batch_size=16, seed=42)
val_mask_generator = mask_datagen.flow(y_val, batch_size=16, seed=42)

In [57]:
def combined_generator(image_gen, mask_gen, labels):
    while True:
        # Retrieve the next batch of images and masks
        images = next(image_gen)[0]  # (batch_size, 224, 224, 3)
        masks = next(mask_gen)[0]    # (batch_size, 224, 224, 1)
        
        # Ensure the batch sizes match
        batch_size = min(len(images), len(masks))
        images = images[:batch_size]
        masks = masks[:batch_size]
        label_batch = labels[:batch_size]  # Ensure we have the right batch size for labels
        
        # Yield images as the single input, and a dictionary of outputs
        yield images, {"segmentation_output": masks, "classification_output": label_batch}

In [49]:
BACKBONE = 'mobilenetv2'
unet_model = Unet(BACKBONE, input_shape=(224, 224, 3), classes=1, activation='sigmoid', encoder_weights='imagenet')

# Segmentation output from U-Net
segmentation_output = unet_model.output

# Classification branch - take features from the encoder
# Get the output of the last layer of the encoder (MobileNet)
encoder_output = unet_model.get_layer("final_conv").output  # Adjust layer name if needed


AttributeError: 'function' object has no attribute 'images'

In [50]:
# Add global pooling and dense layers for 7-class classification
x = GlobalAveragePooling2D()(encoder_output)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
classification_output = Dense(7, activation='softmax')(x)

In [58]:
# Define the final model with two outputs
model = Model(inputs=unet_model.input, outputs=[segmentation_output, classification_output])

# Compile the model with appropriate losses and metrics for each output
model.compile(
    optimizer=Adam(lr=0.0001),
    loss={'output_1': 'binary_crossentropy', 'output_2': 'sparse_categorical_crossentropy'},
    metrics={'output_1': [iou_score], 'output_2': ['accuracy']}
)

print(model.summary())

Model: "model_11"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1_pad (ZeroPadding2D)      (None, 225, 225, 3)  0           ['input_2[0][0]']                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['Conv1_pad[0][0]']              
                                )                                                                 
                                                                                           

c:\Users\Sarthak\anaconda3\envs\ML_GPU\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


 block_6_depthwise (DepthwiseCo  (None, 14, 14, 192)  1728       ['block_6_pad[0][0]']            
 nv2D)                                                                                            
                                                                                                  
 block_6_depthwise_BN (BatchNor  (None, 14, 14, 192)  768        ['block_6_depthwise[0][0]']      
 malization)                                                                                      
                                                                                                  
 block_6_depthwise_relu (ReLU)  (None, 14, 14, 192)  0           ['block_6_depthwise_BN[0][0]']   
                                                                                                  
 block_6_project (Conv2D)       (None, 14, 14, 64)   12288       ['block_6_depthwise_relu[0][0]'] 
                                                                                                  
 block_6_p

In [45]:
train_mask_generator.x.shape

(8012, 224, 224, 1)

In [55]:
# Train the model
history = model.fit(
    combined_generator(train_image_generator, train_mask_generator, labels_train),  
    steps_per_epoch=len(X_train) // 16,
    validation_data=combined_generator(val_image_generator, val_mask_generator, labels_val),
    validation_steps=len(X_val) // 16,
    epochs=50
)

# Save the model
model.save('ham10000_unet_7class_classification.h5')

Epoch 1/50


ValueError: in user code:

    File "c:\Users\Sarthak\anaconda3\envs\ML_GPU\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\Sarthak\anaconda3\envs\ML_GPU\lib\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\Sarthak\anaconda3\envs\ML_GPU\lib\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\Sarthak\anaconda3\envs\ML_GPU\lib\site-packages\keras\engine\training.py", line 993, in train_step
        y_pred = self(x, training=True)
    File "c:\Users\Sarthak\anaconda3\envs\ML_GPU\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\Users\Sarthak\anaconda3\envs\ML_GPU\lib\site-packages\keras\engine\input_spec.py", line 232

In [59]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Concatenate, Input
from tensorflow.keras.models import Model

def build_model(input_shape):
    base_model = EfficientNetB0(include_top=False, weights='imagenet', input_shape=input_shape)

    image_input = Input(shape=input_shape)
    mask_input = Input(shape=(input_shape[0], input_shape[1], 1))

    # Image Branch
    x = base_model(image_input, training=False)
    x = GlobalAveragePooling2D()(x)

    # Mask Branch
    y = GlobalAveragePooling2D()(mask_input)

    # Combine both
    combined = Concatenate()([x, y])
    combined = Dense(256, activation='relu')(combined)
    combined = Dense(128, activation='relu')(combined)
    output = Dense(7, activation='softmax')(combined)  # Assuming 7 classes for skin lesions

    model = Model(inputs=[image_input, mask_input], outputs=output)

    return model

model = build_model((224, 224, 3))
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

16705208/16705208 [==============================] - 25s 1us/step


In [ ]:
batch_size = 32
epochs = 50

train_gen = datagen.flow([X_train, masks_train], y_train, batch_size=batch_size)

history = model.fit(
    train_gen,
    steps_per_epoch=len(X_train) // batch_size,
    epochs=epochs,
    validation_data=([X_test, masks_test], y_test)
)
